### Project 6 - Description

The goal of this project is to rewrite the pull pipeline we created in the **Application - Pipelines - Pulling** video in the **Generators as Coroutines** section.

You should look at the techniques we used in the **Application - Pipelines - Broadcasting** video and apply them here.

The goal is to write a pipeline that will push data from the source file, `cars.csv`, and push it through some filters and a save coroutine to ultimately save the results as a csv file.

Try to make your code as generic as possible, and don't worry about column headers in the output file (unless you really want to!).

When you are done with your solution you should be able to specify an arbitrary number of filters on the name field.

If you specify `Chevrolet`, `Carlo` and `Landau` for three filters, your output file should contain two lines of data only:

```
Chevrolet Monte Carlo Landau,15.5,8,350.0,170.0,4165.,11.4,77,US
Chevrolet Monte Carlo Landau,19.2,8,305.0,145.0,3425.,13.2,78,US
```

Good luck!!

In [58]:
import csv

def parse_data(f_name):
    f = open(f_name)
    try:
        dialect = csv.Sniffer().sniff(f.read(2000))
        f.seek(0)
        next(f)  # skip header row
        yield from csv.reader(f, dialect=dialect)
    finally:
        f.close()

In [59]:
def coroutine(fn):
    def inner(*args, **kwargs):
        coro = fn(*args, **kwargs)
        next(coro)
        return coro
    return inner

In [60]:
@coroutine
def save_csv(f_name):
    with open(f_name, 'w', newline='') as f:
        writer = csv.writer(f)
        while True:
            row = yield
            writer.writerow(row)


In [61]:
@coroutine
def filter_data(filter_pred, target):
    while True:
        row = yield
        if filter_pred(row):
            target.send(row)


In [62]:
@coroutine
def pipeline_coro(out_file, name_filters):
    save = save_csv(out_file)
    
    target = save
    for name_filter in name_filters:
        target = filter_data(lambda d, v=name_filter: v in d[0], target)
        # warning: we have to use the trick above because
        # lambdas are actually closures and the free variable name_filter
        # is a shared free variable - we have seen this problem before!
    while True:
        received = yield
        target.send(received)

In [63]:
from contextlib import contextmanager

@contextmanager
def pipeline(out_file, name_filters):
    p = pipeline_coro(out_file, name_filters)
    try:
        yield p
    finally:
        p.close()

In [64]:
with pipeline('out.csv', ('Chevrolet', 'Landua', 'Carlo')) as p:
    for row in parse_data('cars.csv'):
        p.send(row)

In [66]:
with open('out.csv') as f:
    for row in f:
        print(row)